In [1]:
!pip install transformers datasets tokenizers
!wget  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip #-qq flag indicate quiet mode where it will suppress all the output except for error message
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
import os
import torch, re, random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import numpy as np, math, itertools
import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import Adam
from pathlib import Path

## Tokenization:


## Pre-processing:


In [3]:
class BERTDataset(Dataset):
  def __init__(self, data_pair, tokenizer, seq_len=64):
    self.tokenizer=tokenizer
    self.seq_len=seq_len
    self.corpus_lines=len(data_pair)
    self.lines=data_pair

  def __len__(self):
    return self.corpus_lines

  def __getitem__(self, item):
    #getting random sentence pair(saved as is_next_label)
    t1, t2, is_next_label=self.get_sent(item)

    #replacing random words in sentence with mask/random words:
    t1_random, t1_label=self.random_word(t1)
    t2_random, t2_label=self.random_word(t2)

    #adding special tokens:
    t1=[self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]
    t2=t2_random + [self.tokenizer.vocab['[SEP]']]
    t1_label=[self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]
    t2_label +=[self.tokenizer.vocab['[PAD]']]

    #combining sentence one and two as one input and adding PAD tokens
    #to make sentence same length as seq_len
    segment_label=([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
    bert_input=(t1+t2)[:self.seq_len]
    bert_label=(t1_label + t2_label)[:self.seq_len]
    padding=[self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
    bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)

    output={
        "bert_input": bert_input,
        "bert_label":bert_label,
        "segment_label": segment_label,
        "is_next": is_next_label
    }

    return {key: torch.tensor(value) for key, value in output.items()}

  def random_word(self, sentence):
    tokens=sentence.split()
    output=[] #stores modified tokens id
    output_label=[] # stores original token id

    # according to the bert paper, 15% of the tokens are generally replaced:
    for  i, token in enumerate(tokens):
      prob=random.random() #generate random prob between 0 to 1

      #removing special tokens: [CLS] and [SEP]
      token_id=self.tokenizer(token)['input_ids'][1:-1]

      # 15% token position at random:
      if prob < 0.15:
        prob /= 0.15

        # 80% masking in the selected position:
        if prob < .8:
          for i in range(len(token_id)):
            output.append(self.tokenizer.vocab['[MASK]'])

        # 10% change to random token:
        elif prob < 0.9:
          for i in range(len(token_id)):
            output.append(random.randrange(len(self.tokenizer.vocab)))

        # 10% of the current token:
        else:
          output.append(token_id)

        output_label.append(token_id)

      else:
        output.append(token_id)
        for i in range(len(token_id)):
          output_label.append(0)

    # flattening is done:
    output=list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
    output_label=list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
    assert len(output) == len(output_label)
    return output, output_label

  def get_sent(self, index):
    """return random sentence pair"""
    t1, t2=self.get_corpus_line(index)

    #for next sentence prediction:
    if random.random() > 0.5:
      return t1, t2, 1
    else:
      return t1, self.get_random_line(), 0


  def get_corpus_line(self, item):
    """return sentence pair"""
    return self.lines[item][0], self.lines[item][1]

  def get_random_line(self):
    """return random single sentence"""
    return self.lines[random.randrange(len(self.lines))][1]
